In [ ]:
! dvc init

## Load data

In [1]:
%load_ext dotenv
%dotenv

In [2]:
! dvc get https://github.com/iterative/dataset-registry \
          get-started/data.xml -o data/data.xml
! dvc add data/data.xml

0% Downloading|                                    |0/1 [00:00<?,     ?file/s]
!
  0%|          |get-started/data.xml           0.00/37.9M [00:00<?,       ?it/s]
  0%|          |get-started/data.xml      64.0k/36.1M [00:00<02:45,     228kB/s]
  0%|          |get-started/data.xml       128k/36.1M [00:00<02:44,     230kB/s]
  1%|          |get-started/data.xml       256k/36.1M [00:00<02:07,     296kB/s]
  1%|          |get-started/data.xml       320k/36.1M [00:00<01:52,     333kB/s]
  1%|          |get-started/data.xml       384k/36.1M [00:00<01:43,     362kB/s]
  1%|          |get-started/data.xml       448k/36.1M [00:01<01:38,     382kB/s]
  1%|▏         |get-started/data.xml       512k/36.1M [00:01<01:33,     402kB/s]
  2%|▏         |get-started/data.xml       576k/36.1M [00:01<01:29,     419kB/s]
  2%|▏         |get-started/data.xml       640k/36.1M [00:01<01:26,     430kB/s]
  2%|▏         |get-started/data.xml       704k/36.1M [00:01<01:24,     439kB/s]
  2%|▏         |get-started/

## Create a pipeline with some steps. i.e.:
    *  prepare data
    *  turn data into features
    *  train models from features
    *  evaluate models

In [3]:
! dvc run -f -n prepare \
                     -p prepare.seed,prepare.split \
                     -d src/prepare.py -d data/data.xml \
                     -o data/prepared \
                     python src/prepare.py data/data.xml

Stage is cached, skipping


In [4]:
! dvc run -f -n featurize \
          -p featurize.max_features,featurize.ngrams \
          -d src/featurization.py -d data/prepared \
          -o data/features \
          python src/featurization.py data/prepared data/features

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Restored stage 'featurize' from run-cache
Skipping run, checking out outputs
  0% Saving features|                          |0.00/2.00 [00:00<?,     ?file/s]Modifying stage 'featurize' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml


In [5]:
! dvc run -f -n train \
          -p train.seed,train.n_estimators \
          -d src/train.py -d data/features \
          -o model.pkl \
          python src/train.py data/features model.pkl

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Restored stage 'train' from run-cache
Skipping run, checking out outputs
Modifying stage 'train' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml


In [6]:
! dvc run -f -n evaluate \
          -d src/evaluate.py -d model.pkl -d data/features \
          -M scores.json \
          --plots-no-cache prc.json \
          python src/evaluate.py model.pkl \
                 data/features scores.json prc.json

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage is cached, skipping


In [7]:
! dvc repro

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


In [8]:
! dvc remote add -df myremote gs://dvc_intro
! git add .
! git commit -m 'run'
! git push origin data_pipelines

Setting 'myremote' as a default remote.
[data_pipelines 2020cf3] run
 1 file changed, 3 insertions(+), 3 deletions(-)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 501 bytes | 501.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:NichitaDiaconu/dvc_intro.git
   85be72d..2020cf3  data_pipelines -> data_pipelines


In [9]:
! cat scores.json

{"auc": 0.5417487597055675}

In [10]:
! dvc push

0% Analysing status|                         |0.00/6.00 [00:00<?,     ?file/s]Everything is up to date.


In [11]:
! dvc dag

7=






















    +-------------------+  
    | data/data.xml.dvc |  
    +-------------------+  
              *            
              *            
              *            
         +---------+       
         | prepare |       
         +---------+       
              *            
              *            
              *            
        +-----------+      
        | featurize |      
        +-----------+      
         **        **      
       **            *     
      *               **   
+-------+               *  
| train |             **   
+-------+            *     
         **        **      
           **    **        
/tmp/tmpoev5vrtg

In [15]:
! rm -rf data/features
! rm -rf data/prepared
! rm -rf data/data.xml
! rm -rf model.pkl
! rm -rf .dvc/cache
! rm -rf .dvc/tmp